In [33]:
from onnxruntime import InferenceSession
import polars as pl
import numpy as np
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
from hydra import compose, initialize
from onnx import load_model

In [34]:
with initialize(version_base="1.3.2",
                config_path="../src/config",
                job_name="explainer_dashboard"):
    cfg = compose(config_name="config")

In [44]:
cfg.paths.data.X_test_transformed

'/Users/zacklarsen/Documents/Projects/kaggle-wids-datathon-2020/data/X_test_transformed.parquet'

In [43]:
X_test_transformed = pl.read_parquet(cfg.paths.data.X_test_transformed)

In [37]:
y_test = pl.read_parquet(cfg.paths.data.y_test)

In [45]:
X_test_transformed = X_test_transformed.to_pandas().replace('NA', np.nan)

In [46]:
X_test_transformed.head()

,num__pre_icu_los_days,cat__gender_F,cat__gender_M,cat__gender_None,cat__ethnicity_African American,cat__ethnicity_Asian,cat__ethnicity_Caucasian,cat__ethnicity_Hispanic,cat__ethnicity_Native American,cat__ethnicity_Other/Unknown,...,cat__hospital_admit_source_PACU,cat__hospital_admit_source_Recovery Room,cat__hospital_admit_source_Step-Down Unit (SDU),cat__hospital_admit_source_None,cat__icu_admit_source_Accident & Emergency,cat__icu_admit_source_Floor,cat__icu_admit_source_Operating Room / Recovery,cat__icu_admit_source_Other Hospital,cat__icu_admit_source_Other ICU,cat__icu_admit_source_None
0,-0.192791,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,-0.310545,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.457616,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.302836,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.214056,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [40]:
with open(cfg.paths.models.lr, "rb") as f:
    loaded_onnx_model = load_model(f)

In [41]:
sess = InferenceSession(loaded_onnx_model.SerializeToString(), providers=["CPUExecutionProvider"])

In [47]:
pred_ort = sess.run(None, {"X": X_test_transformed.astype(np.float32)})[0]

ValueError: Required inputs (['float_input']) are missing from input feed (['X']).

In [48]:
pred_skl = loaded_onnx_model.predict(X_test_transformed.astype(np.float32))

AttributeError: 'ModelProto' object has no attribute 'predict'

In [50]:
explainer = ClassifierExplainer(loaded_onnx_model, X_test_transformed, y_test.to_pandas(), shap='linear')

AssertionError: for ClassifierExplainer, model should be a scikit-learn compatible *classifier* model that has a predict_proba(...) method, so not a <class 'onnx.onnx_ml_pb2.ModelProto'>! If you are using e.g an SVM with hinge loss (which does not support predict_proba), you can try the following monkey patch:

import types
def predict_proba(self, X):
    pred = self.predict(X)
    return np.array([1-pred, pred]).T
model.predict_proba = types.MethodType(predict_proba, model)


In [ ]:
ExplainerDashboard(explainer).run()